In [54]:
import json
import os
import sys

import boto3
import asyncio
from typing import List

from langchain_community.llms import Bedrock
from langchain_community.chat_models import BedrockChat

from langchain.prompts import ChatPromptTemplate, PromptTemplate

from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator
#from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import GithubFileLoader
from langchain_core.runnables import RunnablePassthrough


model_id = "mistral.mistral-large-2402-v1:0"

model_kwargs = { 
    "max_tokens": 6000
}

bedrock_runtime = boto3.client('bedrock-runtime')

llm = BedrockChat(
    client=bedrock_runtime,
    model_id=model_id,
)

chat = BedrockChat(model_id=model_id)

In [55]:
! pip install asyncio
# parser = StrOutputParser()

In [ ]:
response_schemas = [
    ResponseSchema(name="Output", description="modernize and generate the code in java format"),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="You are a software architect, answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

#print (format_instructions)

In [2]:
# If you haven't set your access token as an environment variable, pass it in here.
from getpass import getpass

ACCESS_TOKEN = getpass()


 ········


In [70]:
loader = GithubFileLoader(
    repo="jihed/helloworld-mdb",
    access_token=ACCESS_TOKEN,  # delete/comment out this argument if you've set the access token as an env var.
    branch= "main",
    github_api_url="https://api.github.com",
    file_filter=lambda file_path: file_path.endswith(
        "HelloWorldTopicMDB.java"
    ),  # load all java files.
)


In [75]:
sample_source_code= loader.load()

In [ ]:
!pip install -qU langchain-text-splitters

In [ ]:
# Full list of supported languages
[e.value for e in Language]

In [ ]:
RecursiveCharacterTextSplitter.get_separators_for_language(Language.JAVA)

In [76]:
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_messages(
    [("system", "Generate for me a step by step guide in mardown format to migrate all the documents in context written in enterprise Java EE source code to Quarkus:\n\n{context}")]
)
chain = create_stuff_documents_chain(llm, prompt)
#print(print(documents))

response = chain.invoke({"context": sample_source_code})

print(response, end= '\n')



# Migrating from Enterprise Java EE to Quarkus

Quarkus is a Kubernetes native Java stack tailored for OpenJDK HotSpot and GraalVM, crafted from the best of breed Java libraries and standards. Here is a step-by-step guide to migrate your Java EE application to Quarkus.

## Prerequisites

- Java 11 or later
- Apache Maven 3.5.x or later
- A basic understanding of Quarkus and Java EE

## Step 1: Set up your Quarkus project

First, you need to set up a new Quarkus project. You can use the Quarkus Maven plugin to generate a new project:

```bash
mvn io.quarkus:quarkus-maven-plugin:1.13.3.Final:create \
    -DprojectGroupId=org.acme \
    -DprojectArtifactId=getting-started \
    -DclassName="org.acme.GettingStartedResource" \
    -Dpath="/hello"
```

## Step 2: Add dependencies

Add the necessary dependencies to your `pom.xml` file. For example, if you are using JMS, you would add:

```xml
<dependency>
    <groupId>io.quarkus</groupId>
    <artifactId>quarkus-narayana-jta</artifactId>
</

# Refine Document
This is an advanced prompt, where we ask the LLM:
1 - Build a Migration Plan from Java EE to Quarkus
2 - Based on that plan, we ask to generate Quarkus code.

// Todo: Add pom.xml to the model

In [84]:
from langchain.chains import RefineDocumentsChain, LLMChain
from langchain_core.prompts import PromptTemplate

# This controls how each document will be formatted. Specifically,
# it will be passed to `format_document` - see that function for more
# details.
document_prompt = PromptTemplate(
    input_variables=["page_content"],
     template="{page_content}"
)
document_variable_name = "context"

# The prompt here should take as an input variable the
# `document_variable_name`
prompt = PromptTemplate.from_template(
    "Build a migration plan from Java Entreprise EE to Quarkus for this content: {context}"
)
initial_llm_chain = LLMChain(llm=llm, prompt=prompt)
initial_response_name = "prev_response"
# The prompt here should take as an input variable the
# `document_variable_name` as well as `initial_response_name`
prompt_refine = PromptTemplate.from_template(
    "Here's your Migration Plan: {prev_response}. "
    "Now generate a quarkus code for: {context}"
)
refine_llm_chain = LLMChain(llm=llm, prompt=prompt_refine)
chain = RefineDocumentsChain(
    initial_llm_chain=initial_llm_chain,
    refine_llm_chain=refine_llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
    initial_response_name=initial_response_name,
)



response = chain.invoke(sample_source_code)



In [110]:
values = response.values()
keys = response.keys()
#print(keys)

print(response.get('output_text'), end= '\n')

 To migrate the given Java Enterprise Edition (EE) code to Quarkus, follow these steps:

1. Replace the Java EE import statements with their corresponding Quarkus equivalents.
2. Update the `@MessageDriven` annotation with the Quarkus equivalent, `@Incoming`, and configure it for a JMS topic.
3. Replace the `MessageListener` interface with the Quarkus equivalent, which is a method annotated with `@Consume`.
4. Update the logging mechanism to use Quarkus' built-in logger, `io.quarkus.logging.Log`.

Here's the updated code for Quarkus:

```java
package org.jboss.as.quickstarts.mdb;

import io.quarkus.logging.Log;
import jakarta.enterprise.context.ApplicationScoped;
import jakarta.jms.JMSException;
import jakarta.jms.Message;
import jakarta.jms.TextMessage;
import jakarta.jms.Topic;

import org.eclipse.microprofile.reactive.messaging.Channel;
import org.eclipse.microprofile.reactive.messaging.Incoming;
import org.eclipse.microprofile.reactive.messaging.MessageHandler;

@ApplicationScoped
